In [8]:
import sqlite3
import random
import requests
import os
import zipfile
import concurrent.futures
import datetime

# Connect to the SQLite database or create it if it doesn't exist
conn = sqlite3.connect('names.db')
cursor = conn.cursor()

# Create the table with ID, FirstName, LastName, and timestamp including milliseconds
cursor.execute('''
CREATE TABLE IF NOT EXISTS Names (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    FirstName TEXT NOT NULL,
    LastName TEXT NOT NULL,
    Timestamp TEXT DEFAULT (STRFTIME('%Y-%m-%d %H:%M:%f', 'NOW'))
)
''')

conn.commit()

# List of names to use for generation
names_list = ['riley', 'james', 'ryan', 'joshi', 'andrew']

# Function to generate a unique name pair
def generate_unique_name(existing_names):
    while True:
        first_name = random.choice(names_list)
        last_name = random.choice(names_list)
        if first_name != last_name:
            name_pair = (first_name, last_name)
            if name_pair not in existing_names:
                existing_names.add(name_pair)
                return name_pair

# Function to insert a name pair into the database
def insert_name_pair(name_pair):
    try:
        conn = sqlite3.connect('names.db')
        cursor = conn.cursor()
        cursor.execute('''
        INSERT INTO Names (FirstName, LastName)
        VALUES (?, ?)
        ''', name_pair)
        conn.commit()
    finally:
        conn.close()

# Generate 20 unique name pairs
existing_names = set()
name_pairs = [generate_unique_name(existing_names) for _ in range(20)]

# Multiply by 50 to have 1000
name_pairs = name_pairs * 50

# Insert name pairs into the database in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(insert_name_pair, name_pairs)

# Retrieve all rows from the database
conn = sqlite3.connect('names.db')
cursor = conn.cursor()
cursor.execute('SELECT ID, FirstName, LastName, Timestamp FROM Names')
rows = cursor.fetchall()
conn.close()

# Create a directory to store the QR code images
if not os.path.exists('qrcodes'):
    os.makedirs('qrcodes')

# Generate QR codes for each row
qr_code_base_url = 'https://image-charts.com/chart'
for row in rows:
    id_value = row[0]
    qr_code_url = f'{qr_code_base_url}?cht=qr&chs=150x150&chl={id_value}'
    qr_code_image = requests.get(qr_code_url).content
    with open(f'qrcodes/{id_value}.png', 'wb') as f:
        f.write(qr_code_image)

# Create a zip file containing all QR code images
with zipfile.ZipFile('qrcodes.zip', 'w') as zipf:
    for root, _, files in os.walk('qrcodes'):
        for file in files:
            zipf.write(os.path.join(root, file), arcname=file)

            
            
# Display all data from the Names table
conn = sqlite3.connect('names.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Names')
for row in cursor:
    print(row)

conn.close()


(1, 'james', 'andrew', '2024-06-14 11:00:12.061')
(2, 'ryan', 'james', '2024-06-14 11:00:12.083')
(3, 'andrew', 'james', '2024-06-14 11:00:12.086')
(4, 'james', 'ryan', '2024-06-14 11:00:12.090')
(5, 'joshi', 'ryan', '2024-06-14 11:00:12.095')
(6, 'james', 'joshi', '2024-06-14 11:00:12.099')
(7, 'ryan', 'joshi', '2024-06-14 11:00:12.103')
(8, 'james', 'riley', '2024-06-14 11:00:12.107')
(9, 'andrew', 'ryan', '2024-06-14 11:00:12.111')
(10, 'ryan', 'riley', '2024-06-14 11:00:12.130')
(11, 'riley', 'andrew', '2024-06-14 11:00:12.134')
(12, 'joshi', 'james', '2024-06-14 11:00:12.142')
(13, 'joshi', 'riley', '2024-06-14 11:00:12.157')
(14, 'joshi', 'andrew', '2024-06-14 11:00:12.173')
(15, 'andrew', 'riley', '2024-06-14 11:00:12.188')
(16, 'ryan', 'andrew', '2024-06-14 11:00:12.203')
(17, 'riley', 'joshi', '2024-06-14 11:00:12.219')
(18, 'andrew', 'joshi', '2024-06-14 11:00:12.234')
(19, 'riley', 'ryan', '2024-06-14 11:00:12.250')
(20, 'riley', 'james', '2024-06-14 11:00:12.282')
(21, 'ril